In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [24]:
# Declare the lists for each feature to be scraped
repo_url = []
repo_watches = []
repo_sponsors = []
repo_open_issues = []
repo_closed_issues = []
repo_labels = []
repo_milestones = []
repo_open_prs = []
repo_closed_prs = []

In [25]:
def scrape_page(project_url):
    # Add url to list
    repo_url.append(project_url)

    r = requests.get(project_url)

    # Check if the request was successful (status code 200)
    if r.status_code == 200:
        print(f"Parsing {project_url}")
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(r.text, 'html.parser')

        #print(soup.prettify())
        
        # Get the OWNER/REPO
        project = project_url[19:]
        
        # Parse HTML
        # Get number of watches and sponsered?
        num_watches = soup.find(href=f"/{project}/watchers").find("strong").text
        
        creator = project.split('/')[0]
        sponsored = "Yes" if soup.find(href=f"/sponsors/{creator}") != None else "No"
        
        repo_watches.append(num_watches)
        repo_sponsors.append(sponsored)
        
        
        # Issues
        issue_url = project_url + "/issues"
        r = requests.get(issue_url)
        soup = BeautifulSoup(r.text,"html.parser")
        
        open_issues = soup.find(href=f"/{project}/issues?q=is%3Aopen+is%3Aissue").text.split()[0]
        closed_issues = soup.find(href=f"/{project}/issues?q=is%3Aissue+is%3Aclosed").text.split()[0]
        num_labels = soup.find(href=f"/{project}/labels").find("span").text
        num_milestones = soup.find(href=f"/{project}/milestones").find("span").text
        
        repo_open_issues.append(open_issues)
        repo_closed_issues.append(closed_issues)
        repo_labels.append(num_labels)
        repo_milestones.append(num_milestones)
        
        
        # Pull Requests
        pull_url = project_url + "/pulls"
        r = requests.get(pull_url)
        soup = BeautifulSoup(r.text,"html.parser")
        
        open_prs = soup.find(href=f"/{project}/pulls?q=is%3Aopen+is%3Apr").text.split()[0]
        closed_prs = soup.find(href=f"/{project}/pulls?q=is%3Apr+is%3Aclosed").text.split()[0]
        
        repo_open_prs.append(open_prs)
        repo_closed_prs.append(closed_prs)
        
    else:
        print(f"Failed to retrieve data. Status code: {r.status}")
    

In [26]:
project_list = ["https://github.com/twbs/bootstrap", "https://github.com/freeCodeCamp/freeCodeCamp", "https://github.com/geekyutao/Inpaint-Anything", "https://github.com/spring-attic/spring-mvc-showcase", "https://github.com/raspberrypi/firmware", "https://github.com/negomi/react-burger-menu", "https://github.com/yyhsong/iDataV", "https://github.com/yipianfengye/android-zxingLibrary", "https://github.com/nikic/FastRoute", "https://github.com/vercel/platforms", "https://github.com/thinkingjimmy/Learning-Prompt"]
for project in project_list:
    scrape_page(project)

Parsing https://github.com/twbs/bootstrap


AttributeError: 'NoneType' object has no attribute 'text'

In [10]:
projects_df = pd.DataFrame({'Project URL':repo_url, 
                            'Number of Watches':repo_watches,
                            'Sponsored':repo_sponsors,
                            'Open Issues':repo_open_issues,
                            'Closed Issues':repo_closed_issues,
                            'Number of Labels':repo_labels,
                            'Number of Milestones':repo_milestones,
                            'Open Pull Requests':repo_open_prs,
                            'Closed Pull Requests':repo_closed_prs})

In [11]:
projects_df

,Project URL,Number of Watches,Sponsored,Open Issues,Closed Issues,Number of Labels,Number of Milestones,Open Pull Requests,Closed Pull Requests
0,https://github.com/twbs/bootstrap,6.8k,No,394,"21,901",60,0,138,"14,899"
1,https://github.com/freeCodeCamp/freeCodeCamp,8.5k,No,270,"17,414",44,4,54,"35,527"
